In [4]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from operator import itemgetter, attrgetter
import random
import pickle

import importlib
import sn_graph_functions as gf
importlib.reload(gf)
import predictor as pred
importlib.reload(pred)

<module 'predictor' from 'G:\\Develop\\Repositories\\VK_test\\version3\\predictor.py'>

In [5]:
initial_graph = gf.create_initial_graph("data/friends_dataset")

In [6]:
print(nx.info(initial_graph))

Name: 
Type: DiGraph
Number of nodes: 43895
Number of edges: 1035818
Average in degree:  23.5976
Average out degree:  23.5976


In [7]:
%%time
train_graph, train_edges_negative, test_edges_positive = gf.create_train_test(initial_graph)

Wall time: 4min 47s


In [9]:
nx.write_gpickle(train_graph, "data/train_graph")

In [11]:
pred.save_to_pickle(train_edges_negative, "data/train_edges_negative")
pred.save_to_pickle(test_edges_positive, "data/test_edges_positive")

In [12]:
gr = train_graph.copy()
undir_g = gr.to_undirected()

In [13]:
%%time

edges1 = list(set(gr.edges()))
data1 = pred.create_features(edges=edges1, g=gr, undir_g=undir_g, add_answers=True, initial_graph=initial_graph)

edges2 = list(set(train_edges_negative))
data2 = pred.create_features(edges=edges2, g=gr, undir_g=undir_g, add_answers=True, initial_graph=initial_graph)

edges3 = list(set(test_edges_positive))
data3 = pred.create_features(edges=edges3, g=gr, undir_g=undir_g, add_answers=True, initial_graph=initial_graph)

train_data_shuffled = np.concatenate((data1, data2), axis=0)
np.random.shuffle(train_data_shuffled)
train_data_shuffled.shape

Wall time: 59min 59s


(1570328, 10)

In [14]:
pred.save_to_pickle(data1, "data/features_train_graph")
pred.save_to_pickle(data2, "data/features_train_edges_negative")
pred.save_to_pickle(data3, "data/test_data")
pred.save_to_pickle(train_data_shuffled, "data/train_data_shuffled")

In [15]:
#initial_graph = gf.create_initial_graph("data/friends_dataset")
test_data = pred.load_from_pickle("data/test_data")
train_data = pred.load_from_pickle("data/train_data_shuffled")

In [16]:
test_ids1 = list(set(test_data[:, -3].astype(int)))
test_ids2 = list(set(test_data[:, -2].astype(int)))
test_edges = set(set(zip(test_ids1, test_ids2)))

train_ids1 = train_data[:, -3].astype(int)
train_ids2 = train_data[:, -2].astype(int)
train_edges = set(set(zip(train_ids1, train_ids2)).union(set(zip(train_ids2, train_ids1))))

In [17]:
%%time
test_edges_negative = []
for id1 in test_ids1:
    i=0
    for id2 in set(gf.friends_of_friends(id1, initial_graph)):
        if id2 in set(nx.non_neighbors(initial_graph, id1)):
            if (id1, id2) not in train_edges:
                test_edges_negative.append((id1, id2))
                i += 1
            if len(test_edges_negative)>=len(test_ids1):
                break
            if i>=20:
                break
        if i>=20:
                break
    if len(test_edges_negative)>=len(test_ids1):
        break
    

Wall time: 1min 52s


In [61]:
#train_graph = nx.read_gpickle("data/train_graph")
# gr = train_graph.copy()
# undir_g = gr.to_undirected()

In [18]:
%%time
features_test_edges_negative = pred.create_features(edges=test_edges_negative, g=gr, undir_g=undir_g, add_answers=True, initial_graph=initial_graph)

Wall time: 9.14 s


In [19]:
pred.save_to_pickle(features_test_edges_negative, "data/features_test_edges_negative")
test_data_with_negatives = np.concatenate((test_data, features_test_edges_negative), axis=0)
pred.save_to_pickle(test_data_with_negatives, "data/test_data_with_negatives")
test_data_with_negatives.shape

(67007, 10)

In [20]:
print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 43895
Number of edges: 982305
Average in degree:  22.3785
Average out degree:  22.3785


In [21]:
print(len(train_edges_negative), len(test_edges_positive))

588023 54359


In [23]:
train_data[train_data[:,-1]==0].shape

(588023, 10)

In [25]:
train_data[train_data[:,-1]==1].shape

(982305, 10)